In [1]:
import numpy as np
import math
import plotly.graph_objects as go
from scipy.integrate import odeint
from quaternion import Quaternion
from plotly.subplots import make_subplots
import plotly.express as px


#from math import

In [2]:
def sphere(x, y, z, radius, resolution=20, color='#325bff'):
    """Return the coordinates for plotting a sphere centered at (x,y,z)"""
    u, v = np.mgrid[0:2*np.pi:resolution*2j, 0:np.pi:resolution*1j]
    X = radius * np.cos(u)*np.sin(v) + x
    Y = radius * np.sin(u)*np.sin(v) + y
    Z = radius * np.cos(v) + z
    return go.Surface(x=X, y=Y, z=Z, opacity=0.5, colorscale=[[0,color], [1,color]])

def create_orbit(X, Y, Z, clr='black', wdth=2):
    trace = go.Scatter3d(x=X, y=Y, z=Z, marker=dict(size=0.1), line=dict(color=clr,width=wdth))
    return trace

In [13]:
class System:
    # initial parameters
    mu = 3.986004418E+05  # Earth's gravitational parameter [km^3/s^2]
    r_Earth = 6378  # Average radius of Earth [km]
    R = 6400
    omega = np.sqrt(mu/np.power(R, 3))

    # Same
    X_0 = np.array([0, 6400, 0])  # [km]
    V_0 = np.array([omega * R + 0.1, 0, 0])  # [km/s]

    # Nearby
    # X_0 = np.array([0, 9000.1, 0])  # [km]
    # V_0 = np.array([omega * R + 0.2, 0, 0])  # [km/s]

    # Away z
    # X_0 = np.array([0, 9005, 0])  # [km]
    # V_0 = np.array([8.300082319, 0, 0])  # [km/s]

    # Away z
    # X_0 = np.array([0, 9005, 0])  # [km]
    # V_0 = np.array([9.700082319, 0, 5])  # [km/s]

    state_0 = np.concatenate([X_0, V_0])

    def __init__(self, t = 6): # [h]
        self.t = np.linspace(0, t*3600, 200)
        r1_vec = np.array([0, self.R, 0])
        omega_vec = np.array([0, 0, -self.omega])
        r_vec = np.array(self.X_0) - np.array([0, self.R, 0])
        self.X_0_relative = self.X_0 - r1_vec
        self.V_0_relative = self.V_0
        self.V_0_relative -= np.array([self.omega * self.R, 0, 0])
        self.V_0_relative -= np.cross(omega_vec, r_vec)
        self.state_0_relative = np.concatenate([self.X_0_relative, self.V_0_relative])

        self.quaternion = []
        for t in self.t:
            tmp_q = Quaternion(a = self.omega * t, n = np.array([0., 0., -1.]))
            self.quaternion.append(tmp_q)
        self.quaternion = np.array(self.quaternion)


    def __calculate_sat_2_orbit_abs__(self):
        def equation(state_0, t):
            x = state_0[0]
            y = state_0[1]
            z = state_0[2]
            x_dot = state_0[3]
            y_dot = state_0[4]
            z_dot = state_0[5]
            x_ddot = -self.mu * x / (x ** 2 + y ** 2 + z ** 2) ** (3 / 2)
            y_ddot = -self.mu * y / (x ** 2 + y ** 2 + z ** 2) ** (3 / 2)
            z_ddot = -self.mu * z / (x ** 2 + y ** 2 + z ** 2) ** (3 / 2)
            dstate_dt = np.array([x_dot, y_dot, z_dot, x_ddot, y_ddot, z_ddot])
            return dstate_dt

        sol = odeint(equation, self.state_0, self.t)
        tmp_x = sol[:, 0]  # X-coord [km] of satellite over time interval
        tmp_y = sol[:, 1]  # Y-coord [km] of satellite over time interval
        tmp_z = sol[:, 2]  # Z-coord [km] of satellite over time interval
        return np.array([tmp_x, tmp_y, tmp_z]).transpose()

    def __calculate_sat_2_orbit_relative__(self):
        states_abs = self.__calculate_sat_2_orbit_abs__()
        for t in range(len(self.t)):
            states_abs[t] = self.quaternion[t].inverse.rotatePoint(states_abs[t]) - np.array([0, self.R, 0])
        return states_abs

    def __calculate_sat_2_orbit_relative_linear__(self):
        matrix = np.empty([6, 6, len(self.t)])

        matrix[0,0] = np.ones_like(self.t)
        matrix[0,1] = 6*(np.sin(self.omega * self.t) - self.omega * self.t)
        matrix[0,2] = np.ones_like(self.t)
        matrix[0,3] = 4/self.omega * np.sin(self.omega * self.t) - 3 * self.t
        matrix[0,4] = 2/self.omega * (np.cos(self.omega * self.t) - 1)
        matrix[0,5] = np.ones_like(self.t)

        matrix[1,0] = np.zeros_like(self.t)
        matrix[1,1] = 4 * np.ones_like(self.t) - 3 * np.cos(self.omega * self.t)
        matrix[1,2] = np.ones_like(self.t)
        matrix[1,3] = 2/self.omega * (1 - np.cos(self.omega * self.t))
        matrix[1,4] = np.sin(self.omega * self.t)/self.omega
        matrix[1,5] = np.ones_like(self.t)

        matrix[2,0] = np.zeros_like(self.t)
        matrix[2,1] = np.zeros_like(self.t)
        matrix[2,2] = np.cos(self.omega * self.t)
        matrix[2,3] = np.zeros_like(self.t)
        matrix[2,4] = np.zeros_like(self.t)
        matrix[2,5] = np.sin(self.omega * self.t) / self.omega

        matrix[3,0] = np.zeros_like(self.t)
        matrix[3,1] = 6*self.omega * (np.cos(
            self.omega * self.t) - np.ones_like(self.t))
        matrix[3,2] = np.zeros_like(self.t)
        matrix[3,3] = 4 * np.cos(self.omega * self.t) - 3
        matrix[3,4] = -2 * np.sin(self.omega * self.t)
        matrix[3,5] = np.zeros_like(self.t)

        matrix[4,0] = np.zeros_like(self.t)
        matrix[4,1] = 3 * self.omega * np.sin(self.omega * self.t)
        matrix[4,2] = np.zeros_like(self.t)
        matrix[4,3] = 2 * np.sin(self.omega * self.t)
        matrix[4,4] = np.cos(self.omega * self.t)
        matrix[4,5] = np.zeros_like(self.t)

        matrix[5,0] = np.zeros_like(self.t)
        matrix[5,1] = np.zeros_like(self.t)
        matrix[5,2] = -self.omega * np.sin(self.omega * self.t)
        matrix[5,3] = np.zeros_like(self.t)
        matrix[5,4] = np.zeros_like(self.t)
        matrix[5,5] = np.cos(self.omega * self.t)

        matrix = np.transpose(matrix, (2, 0, 1))

        states = np.empty((len(self.t), 6))
        for i in range(len(matrix)):
            states[i] = matrix[i] @ self.state_0_relative
        return np.array([states[:, :3]]).squeeze()

    def __calculate_sat_1_orbit_abs__(self):
        states_abs = []
        for t in range(len(self.t)):
            states_abs.append(self.quaternion[t].rotatePoint(np.array([0, self.R, 0])))
        return np.array(states_abs)

    def __calculate_earth_orbit_relative__(self):
        states_abs = []
        for t in range(len(self.t)):
            states_abs.append(self.quaternion[t].inverse
                              .rotatePoint(np.array([0, -self.R, 0])))
        return np.array(states_abs)

    def GenerateObjectsAbs(self):
        sat1_orbit_states = self.__calculate_sat_1_orbit_abs__()
        sat2_orbit_states = self.__calculate_sat_2_orbit_abs__()
        sat1_orbit = go.Scatter3d(x=sat1_orbit_states[:, 0],
                                y=sat1_orbit_states[:, 1],
                                z=sat1_orbit_states[:, 2], mode='lines',
                                  name='s1_abs',
                                      marker=dict(color='blue'))
        sat2_orbit = go.Scatter3d(x=sat2_orbit_states[:, 0],
                                y=sat2_orbit_states[:, 1],
                                z=sat2_orbit_states[:, 2], mode='lines',
                                  name='s2_abs',
                                      marker=dict(color='red'))
        return [sat1_orbit, sat2_orbit]

    def GenerateObjectsRel(self):
        # earth = sphere(0, 0, 0, self.r_Earth)
        sat2_orbit_states = self.__calculate_sat_2_orbit_relative__()
        sat2_orbit_states_lin = self.__calculate_sat_2_orbit_relative_linear__()
        # earth_orbit = go.Scatter3d(x=earth_orbit_states[:, 0],
        #                         y=earth_orbit_states[:, 1],
        #                         z=earth_orbit_states[:, 2], mode='lines',
        #                            name='Earth relative',
        #                               marker=dict(color='blue'))
        sat2_orbit = go.Scatter3d(x=sat2_orbit_states[:, 0],
                                y=sat2_orbit_states[:, 1],
                                z=sat2_orbit_states[:, 2], mode='lines',
                                  name='S2 relative',
                                      marker=dict(color='red'))
        sat2_orbit_lin = go.Scatter3d(x=sat2_orbit_states_lin[:, 0],
                                      y=sat2_orbit_states_lin[:, 1],
                                      z=sat2_orbit_states_lin[:, 2], mode='lines',
                                      name='S2 relative linear',
                                      marker=dict(color='green'))
        return [sat2_orbit, sat2_orbit_lin]

# s = System(t=0.707)
s = System(t=16)

layout = go.Layout(
             scene=dict(
                 aspectmode='data'
         ), scene_camera = dict(
    up=dict(x=0, y=0, z=0),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0, y=0, z=-40)
))

fig = go.Figure(data = s.GenerateObjectsAbs(), layout=layout)
fig.write_html("modelABS.html")

fig = go.Figure(data = s.GenerateObjectsRel(), layout=layout)
fig.write_html("modelREL.html")
fig = make_subplots(rows=1, cols=2, specs=
                            [[{'type': 'scene'},    {'type': 'scene'}]])
for i in s.GenerateObjectsAbs():
    fig.add_trace(i, row=1, col=1)
for i in s.GenerateObjectsRel():
    fig.add_trace(i, row=1, col=2)
fig.layout['scene1'].update(aspectmode='data', camera = dict(
    up=dict(x=0, y=0, z=0),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0, y=0, z=-40)))
fig.layout['scene2'].update(aspectmode='data', camera = dict(
    up=dict(x=0, y=0, z=0),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0, y=0, z=-40)))
fig.write_html("modelsplit.html")